In [1]:
import itertools
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score

In [2]:
# Load Iris dataset
data = pd.read_csv('/content/drive/MyDrive/Dataset/Iris.csv')
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Encode target labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [3]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Define MLP Model class
def create_mlp(input_size, hidden_layers, activation_fn, num_classes):
    layers = []
    current_size = input_size

    # Add hidden layers
    for layer_size in hidden_layers:
        layers.append(nn.Linear(current_size, layer_size))
        layers.append(activation_fn())
        current_size = layer_size

    # Add output layer
    layers.append(nn.Linear(current_size, num_classes))
    return nn.Sequential(*layers)

# Define parameter grid
hidden_layers_list = [[4], [8], [16], [32], [64], [4, 4], [16, 16], [32, 32], [64, 64], [4, 4, 4]]
activations = [nn.Identity, nn.Sigmoid, nn.ReLU, nn.Tanh]  # Softmax is usually applied to outputs for classification
num_epochs_list = [1, 10, 25, 50, 100, 250]
learning_rates = [10, 1, 0.1, 0.01, 0.001, 0.0001]
batch_sizes = [16, 32, 64, 128, 256, 512]

# Hyperparameter combinations
param_grid = list(itertools.product(hidden_layers_list, activations, num_epochs_list, learning_rates, batch_sizes))

# Train and evaluate MLP model
def train_and_evaluate(hidden_layers, activation_fn, num_epochs, learning_rate, batch_size):
    input_size = X_train.shape[1]
    num_classes = len(np.unique(y))
    model = create_mlp(input_size, hidden_layers, activation_fn, num_classes)

    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in dataloader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluate model
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test_tensor).argmax(axis=1).numpy()
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Test all configurations
results = []
for params in param_grid:
    hidden_layers, activation_fn, num_epochs, learning_rate, batch_size = params
    accuracy = train_and_evaluate(hidden_layers, activation_fn, num_epochs, learning_rate, batch_size)
    results.append((hidden_layers, activation_fn.__name__, num_epochs, learning_rate, batch_size, accuracy))

# Save results
results_df = pd.DataFrame(results, columns=['Hidden Layers', 'Activation', 'Epochs', 'Learning Rate', 'Batch Size', 'Accuracy'])
results_df.to_csv('mlp_iris_results.csv', index=False)

print("Experiment completed. Results saved to 'mlp_iris_results.csv'.")


Experiment completed. Results saved to 'mlp_iris_results.csv'.


In [5]:
import os
print(os.getcwd())

/content


In [7]:
results_df.to_csv('/content/drive/MyDrive/Dataset/mlp_iris_results.csv', index=False)